In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import NN

In [2]:
import dataset_utils
import importlib

In [3]:
importlib.reload(dataset_utils)

<module 'dataset_utils' from 'c:\\Users\\cyber\\Desktop\\University\\P4P\\github\\P4P-mmWave\\MMWave_Radar_Human_Tracking_and_Fall_detection\\dataset_utils.py'>

In [31]:
DATA_DIR = "data/entire_field_raw_3_class"
NUM_POINTS = 100
NUM_CLASSES = 2
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: jumping
processing class: walking


In [32]:
print(test_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [33]:
print(train_points)

[[[ 7.100e-01  4.555e+00  1.299e+00  0.000e+00  1.580e+02]
  [ 7.046e-01  3.730e+00  1.143e+00  0.000e+00  1.310e+02]
  [ 1.992e+00  3.883e+00  1.992e+00  0.000e+00  1.310e+02]
  ...
  [-8.643e-02  2.562e+00  7.344e-01  5.010e-01  2.400e+02]
  [ 7.100e-01  4.555e+00  1.300e+00  0.000e+00  1.630e+02]
  [ 7.100e-01  4.547e+00  1.302e+00  0.000e+00  1.600e+02]]

 [[ 2.770e+00  2.211e+00  4.531e-01 -1.002e+00  1.590e+02]
  [ 1.992e+00  3.881e+00  2.000e+00  0.000e+00  1.330e+02]
  [ 2.174e+00  2.746e+00  1.031e+00  0.000e+00  1.570e+02]
  ...
  [ 2.174e+00  2.744e+00  1.034e+00  0.000e+00  1.740e+02]
  [ 2.174e+00  2.742e+00  1.036e+00  0.000e+00  1.740e+02]
  [ 7.100e-01  4.547e+00  1.302e+00  0.000e+00  1.610e+02]]

 [[ 1.992e+00  3.883e+00  1.999e+00  0.000e+00  1.320e+02]
  [ 2.174e+00  2.744e+00  1.034e+00  0.000e+00  1.740e+02]
  [ 2.174e+00  2.744e+00  1.034e+00  0.000e+00  1.700e+02]
  ...
  [ 7.397e-01  4.527e+00  1.305e+00  0.000e+00  1.550e+02]
  [ 2.466e-01  2.461e+00  1.459e+0

In [34]:
# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()
sequence_length = 100
num_features = 5
num_classes = 3


# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))

# LSTM layer
model.add(layers.LSTM(64, input_shape=(sequence_length, num_features), return_sequences=False))

# Fully connected (Dense) layer
model.add(layers.Dense(32, activation='relu'))

# Output layer with Softmax activation for multi-class classification
model.add(layers.Dense(num_classes, activation='softmax'))




c:\Users\kzho904\.conda\envs\P4P_mmWave\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\kzho904\.conda\envs\P4P_mmWave\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [35]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)               │ (None, None, 64)       │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,227 (141.51 KB)

 Trainable params: 36,227 (141.51 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(train_points, train_labels, epochs=200, validation_data=(test_points, test_labels), verbose=1, callbacks=[early_stopping])

Epoch 1/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5621 - loss: 0.6899 - val_accuracy: 0.5793 - val_loss: 0.6821
Epoch 2/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5943 - loss: 0.6791 - val_accuracy: 0.5793 - val_loss: 0.6812
Epoch 3/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5681 - loss: 0.6854 - val_accuracy: 0.5793 - val_loss: 0.6811
Epoch 4/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5696 - loss: 0.6852 - val_accuracy: 0.5793 - val_loss: 0.6812
Epoch 5/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5718 - loss: 0.6846 - val_accuracy: 0.5793 - val_loss: 0.6810
Epoch 6/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6054 - loss: 0.6737 - val_accuracy: 0.5793 - val_loss: 0.6810
Epoch 7/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5734 - loss: 0.6841 - val_accuracy: 0.5793 - val_loss: 0.6813
Epoch 8/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5767 - loss: 0.6821 - val_accuracy: 0.5793 - 

In [38]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Make predictions on the test set
predictions = model.predict(test_points)

# Convert probabilities to predicted class labels (0, 1, or 2)
predicted_classes = np.argmax(predictions, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(test_labels, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Classification report
class_report = classification_report(test_labels, predicted_classes)
print("Classification Report:")
print(class_report)

# Evaluate the model's accuracy
accuracy = accuracy_score(test_labels, predicted_classes)
print(f"Test Accuracy: {accuracy:.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Confusion Matrix:
[[95  0]
 [69  0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.58      1.00      0.73        95
           1       0.00      0.00      0.00        69

    accuracy                           0.58       164
   macro avg       0.29      0.50      0.37       164
weighted avg       0.34      0.58      0.42       164

Test Accuracy: 0.5793


c:\Users\kzho904\.conda\envs\P4P_mmWave\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kzho904\.conda\envs\P4P_mmWave\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kzho904\.conda\envs\P4P_mmWave\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu